# Toronto Neighbourhoods

## Part 2: Location Data

In part 1, the list of Toronto neighbourhoods has been scraped from the Wikipedia page. This list has been saved as a '.csv' file, which can now be loaded for further enrichment with location data.

In [1]:
import pandas as pd
import numpy as np

import geocoder

In [ ]:
path = '~/Documents/Projects/Coursera-Capstone/Neighbourhoods.csv'
toronto_nbhs_all = pd.read_csv(path)

# Make sure that the dataframe does not already contain the geospatial data.
toronto_nbhs = toronto_nbhs_all[['Postal Code', 'Borough', 'Neighbourhood']]

### Using Geocoder

First an attempt will be made to obtain locatin data using geocoder. In the assignment it has been warned that possibly several attempts need to be made in order to obtain the geodata for a given address. Since 103 addresses need to be enriched, the number of required attempts may exceed the maximum daily allotment.

In [ ]:
# Obtain location data using geocoder, looping until succesful or max allowed attempts is reached.

# define the dataframe columns
column_names = ['Postal Code', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)


for postal_code in toronto_nbhs['Postal Code']:
    neighbourhood_postal_code = postal_code
    lat_lng_coords = None
    
    while lat_lng_coords is None:
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    
    neighbourhood_lat = lat_lng_coords[0]
    neighbourhood_lon = lat_lng_coords[1]

    neighbourhoods = neighbourhoods.append({'Postal Code': neighbourhood_postal_code,
                                            'Latitude': neighbourhood_lat,
                                            'Longitude': neighbourhood_lon}, ignore_index=True)

Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)


In [ ]:
neighbourhoods.head()

### Location data from a '.csv' file

Obtaining location data using Geocoder was found to be too unreliable, since the maximum number of retries was exceeded before resolving the coordinates for all the Toronto neighbourhoods.

As per the instructions, geographical coordinates will now be extracted from a [csv file](https://cocl.us/Geospatial_data).

In [ ]:
path = '~/Documents/Projects/Coursera-Capstone/geodata/Geospatial_Coordinates.csv'
gsd = pd.read_csv(path)

gsd.sort_values(by=['Postal Code'], inplace=True) # Put the list in alphabetical order of the postal codes
gsd.reset_index(inplace=True, drop=True)
print(gsd.shape)
gsd.head()

In [ ]:
# Put the neighbourhood data in the same order as the geospatial coordinates
toronto_nbhs.sort_values(by=['Postal Code'], inplace=True)
toronto_nbhs.reset_index(inplace=True, drop=True)
print(toronto_nbhs.shape)
toronto_nbhs.head()

Now that both dataframes are in the same order, each row refers to the same neighhourhood for both dataframes. The neighbourhoods dataframe (toronto_nbhs) can therefore be simply joined with the 'Latitude' and 'Longitude' columns of the geospatial data dataframe (gsd).

In [ ]:
toronto_nbhs = pd.concat([toronto_nbhs, gsd[['Latitude', 'Longitude']]], axis=1)
toronto_nbhs.head()

In [ ]:
toronto_nbhs.describe(include='all')

This concludes the second part of the assignment.

In [ ]:
# Save the cleaned dataframe as a '.csv'
path = '~/Documents/Projects/Coursera-Capstone/Neighbourhoods.csv'
toronto_nbhs.to_csv(path)